In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
from patchify import patchify, unpatchify
from tensorflow.keras.models import load_model
from skan import Skeleton, summarize
from skimage.morphology import skeletonize
import networkx as nx
import os
import pandas as pd

In [ ]:
Input_dataset = "Input"
Save_results_as = "Results_root_len.csv"

model_location = "Wesley_model.h5"

In [ ]:
def crop_img(img):
    """ Crops an image based on edge detection using Canny.

    Args:
        image_name (str): The filename of the image to be processed.
    
    returns:
        output_image, diff_output_height, diff_output_width, y1, y2, x1, x2
        output_image(ndarray): The cropped image
        diff_output_height(int): the difference in height and width
        diff_output_width(int): the difference in width and height
        y1(int): the upper left corner of the edge
        y2(int): the down left corner of the edge
        x1(int): the upper right corner of the edge
        x2(int): the down right corner of the edge
    """
        

    input_image = img[:, 0:4100]

    canny = cv2.Canny(input_image, 0, 255)

    pts = np.argwhere(canny > 0)
    y1, x1 = pts.min(axis=0)
    y2, x2 = pts.max(axis=0)

    output_image = input_image[y1:y2, x1:x2]
    output_height = output_image.shape[0]
    output_width = output_image.shape[1]
    difference = abs(output_height - output_width)

    if output_height > output_width:
        diff_output_height = output_height - difference
        diff_output_width = output_width
    if output_height < output_width:
        diff_output_width = output_width - difference
        diff_output_height = output_height

    output_image = output_image[0:diff_output_height, 0:diff_output_width]
    return(output_image)

In [ ]:
def error_fix(img):
    """
    Fix errors in an input image.

    This function takes an input image and performs the following operations to fix errors:
    1. Adjust the image dimensions to be square by cropping from the sides if necessary.
    2. Apply the 'crop_img' function twice to further crop the image.

    Parameters:
    img (numpy.ndarray): The input image as a NumPy array.

    Returns:
    numpy.ndarray: The fixed image after error correction.
    """
    diff_shape = round((img.shape[1] - img.shape[0]) / 2)
    diff_shape_1 = img.shape[1] - diff_shape
    img = img[:, diff_shape:diff_shape_1]
    output_img_temp = crop_img(img)
    output_img = crop_img(output_img_temp)
    return(output_img)

In [ ]:
def padder(image, patch_size):
    """
    Adds padding to an image to make its dimensions divisible by a specified patch size.

    This function calculates the amount of padding needed for both the height and width of an image so that its dimensions become divisible by the given patch size. The padding is applied evenly to both sides of each dimension (top and bottom for height, left and right for width). If the padding amount is odd, one extra pixel is added to the bottom or right side. The padding color is set to black (0, 0, 0).

    Parameters:
    - image (numpy.ndarray): The input image as a NumPy array. Expected shape is (height, width, channels).
    - patch_size (int): The patch size to which the image dimensions should be divisible. It's applied to both height and width.

    Returns:
    - numpy.ndarray: The padded image as a NumPy array with the same number of channels as the input. Its dimensions are adjusted to be divisible by the specified patch size.

    Example:
    - padded_image = padder(cv2.imread('example.jpg'), 128)

    """
    h = image.shape[0]
    w = image.shape[1]
    height_padding = ((h // patch_size) + 1) * patch_size - h
    width_padding = ((w // patch_size) + 1) * patch_size - w

    top_padding = int(height_padding/2)
    bottom_padding = height_padding - top_padding

    left_padding = int(width_padding/2)
    right_padding = width_padding - left_padding

    padded_image = cv2.copyMakeBorder(image, top_padding, bottom_padding, left_padding, right_padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    return padded_image

In [ ]:
def f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = TP / (Positives+K.epsilon())
        return recall
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = TP / (Pred_Positives+K.epsilon())
        return precision
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
        total = K.sum(K.square(y_true),[1,2,3]) + K.sum(K.square(y_pred),[1,2,3])
        union = total - intersection
        return (intersection + K.epsilon()) / (union + K.epsilon())
    return K.mean(f(y_true, y_pred), axis=-1)

In [ ]:
def img_predictor(img, root_thr, model_type):
    """
    Predicts root presence in an image using a pre-trained model.

    This function takes an image and applies a pre-trained model to predict root presence. 
    It supports two model types, each with different preprocessing requirements. 
    The image is first padded, then split into patches, which are fed into the model for prediction. 
    The predictions are then reconstructed into a single image. The output is a binary mask representing 
    the predicted root presence and the original (or processed) image.

    Parameters:
    img (ndarray): The input image. Should be a NumPy array, format depending on model_type.
    root_thr (float): Threshold for converting the model's output to a binary mask. 
                      Values above this threshold are considered as root presence.
    model_type (int): Indicator of the model type to use. 
                      1 for the first model, 2 for the second model.

    Returns:
    tuple: A tuple containing:
        - predicted_root (ndarray): A binary mask of the same size as `img`, 
          indicating the presence of roots (1 for root, 0 for no root).
        - img (ndarray): The original or processed image, depending on `model_type`.

    Raises:
    Exception: If `model_type` is not 1 or 2, or other exceptions related to 
               loading models, image processing, or prediction.

    Note:
    This function requires certain external functions and models to be loaded:
    - `load_model` from an external library to load the pre-trained model.
    - `f1`, `iou` custom objects, likely used in the model.
    - `padder`, `patchify`, `unpatchify` for image preprocessing.
    - `cv2` for image manipulation.
    """
    
    if model_type == 1:
        root_mask_model = load_model('task_4_binary/models/all_trained_data_aug2.h5', custom_objects={'f1': f1, 'iou':iou})
    elif model_type == 2:
        root_mask_model = load_model(model_location, custom_objects={'f1': f1, 'iou':iou})
    img = padder(img, 256)

    if model_type == 1:
        patches = patchify(img, (256, 256, 3), step=256)
    if model_type == 2:
        patches = patchify(img, (256, 256), step=256)
    i = patches.shape[0]
    j = patches.shape[1]
    if model_type == 1:
        patches = patches.reshape(-1, 256, 256, 3)
    if model_type == 2:
        patches = patches.reshape(-1, 256, 256, 1)
    preds_root = root_mask_model.predict(patches/255)
    preds_root = preds_root.reshape(i, j, 256, 256)
    predicted_root = unpatchify(preds_root, (img.shape[0], img.shape[1]))
    predicted_root = predicted_root>root_thr
    predicted_root = predicted_root.astype(int)
    predicted_root = cv2.convertScaleAbs(predicted_root) 

    return(predicted_root, img)

In [ ]:
def segmanter(predicted_root, img):
    
    """
    Segments the largest objects in different regions of an image based on a binary mask.

    This function takes a binary mask (predicted_root) indicating the presence of certain objects
    (e.g., roots in an image) and segments the largest objects in predefined regions of the mask. 
    The regions are based on the width of the image. It then extracts these objects from both the 
    binary mask and the original image. The function also plots the segmentation results for visualization.

    Parameters:
    predicted_root (ndarray): A binary mask image, where 1 represents the presence of an object.
    img (ndarray): The original image corresponding to the binary mask.

    Returns:
    tuple: A tuple containing:
        - full_imgs (list of ndarray): Segmented objects from the binary mask.
        - real_imgs (list of ndarray): Corresponding segmented objects from the original image.
        - object_presence (ndarray): A binary array indicating the presence of objects in each region.
        - x (list of int): The x-coordinate of the top-left corner of each segmented object.
        - y (list of int): The y-coordinate of the top-left corner of each segmented object.
        - w (list of int): The width of each segmented object.
        - h (list of int): The height of each segmented object.

    Note:
    - The function splits the image into 5 regions based on width and segments the largest object in each region.
    - It uses `cv2.connectedComponentsWithStats` for segmentation.
    - Plots are created using `matplotlib.pyplot`.
    - This function assumes certain threshold values and dimensions specific to the use case.
    """
     

    label_count, labels, stats, centroids = cv2.connectedComponentsWithStats(predicted_root)

    topmost_coordinates = np.full((label_count, 2), (0, labels.shape[0]), dtype=int)
    
    for y in range(labels.shape[0]):
        for x in range(labels.shape[1]):
            label = labels[y, x]
            if label != 0:  # Ignore the background
                if y < topmost_coordinates[label][1]:
                    topmost_coordinates[label] = [x, y]

    # Remove the entry for the background
    topmost_coordinates = topmost_coordinates[1:]

    large = []
    th_1 = np.sum(predicted_root[300:2500, 0:600]) * 0.1
    th_2 = np.sum(predicted_root[300:2500, 600:1200]) * 0.1
    th_3 = np.sum(predicted_root[300:2500, 1200:1600]) * 0.1
    th_4 = np.sum(predicted_root[300:2500, 1600:2200]) * 0.1
    th_5 = np.sum(predicted_root[300:2500, 2200:]) * 0.1

    image_width = predicted_root.shape[1]

    largest_area_1 = 0
    largest_area_2 = 0
    largest_area_3 = 0
    largest_area_4 = 0
    largest_area_5 = 0

    largest_label_1 = -1
    largest_label_2 = -1
    largest_label_3 = -1
    largest_label_4 = -1
    largest_label_5 = -1

    object_presence = np.zeros(5)


    for x in range(1, label_count):
        if (
            stats[x, cv2.CC_STAT_TOP] < 1500
            and stats[x, cv2.CC_STAT_LEFT] >= 10
            and stats[x, cv2.CC_STAT_LEFT] + stats[x, cv2.CC_STAT_WIDTH] <= image_width - 10
            and stats[x, cv2.CC_STAT_TOP] > 200
            and stats[x, cv2.CC_STAT_AREA] > 100
            
        ):
            if stats[x, cv2.CC_STAT_TOP] < 600 or stats[x, cv2.CC_STAT_AREA] > 3000:
                if (
                    stats[x, cv2.CC_STAT_AREA] > th_1
                    and topmost_coordinates[x-1][0] < 600
                ):
                    area_1 = stats[x, cv2.CC_STAT_AREA]
                    if area_1 > largest_area_1:
                        largest_area_1 = area_1
                        largest_label_1 = x

                if (
                    stats[x, cv2.CC_STAT_AREA] > th_2
                    and topmost_coordinates[x-1][0] > 600
                    and topmost_coordinates[x-1][0] < 1200
                ):
                    area_2 = stats[x, cv2.CC_STAT_AREA]
                    if area_2 > largest_area_2:
                        largest_area_2 = area_2
                        largest_label_2 = x


                if (
                    stats[x, cv2.CC_STAT_AREA] > th_3
                    and topmost_coordinates[x-1][0] > 1200
                    and topmost_coordinates[x-1][0] < 1700
                ):
                    area_3 = stats[x, cv2.CC_STAT_AREA]
                    if area_3 > largest_area_3:
                        largest_area_3 = area_3
                        largest_label_3 = x

                if (
                    stats[x, cv2.CC_STAT_AREA] > th_4
                    and topmost_coordinates[x-1][0] > 1700
                    and topmost_coordinates[x-1][0] < 2300
                ):
                    area_4 = stats[x, cv2.CC_STAT_AREA]
                    if area_4 > largest_area_4:
                        largest_area_4 = area_4
                        largest_label_4 = x

                if (
                    stats[x, cv2.CC_STAT_AREA] > th_5
                    and topmost_coordinates[x-1][0] > 2300
                ):
                    area_5 = stats[x, cv2.CC_STAT_AREA]
                    if area_5 > largest_area_5:
                        largest_area_5 = area_5
                        largest_label_5 = x


    if largest_label_1 != -1:
        large.append(largest_label_1)
        object_presence[0] = 1

    if largest_label_2 != -1:
        large.append(largest_label_2)
        object_presence[1] = 1
        
    if largest_label_3 != -1:
        large.append(largest_label_3)
        object_presence[2] = 1
    
    if largest_label_4 != -1:
        large.append(largest_label_4)
        object_presence[3] = 1

    if largest_label_5 != -1:
        large.append(largest_label_5)
        object_presence[4] = 1

    black_img1 = np.zeros_like(labels, dtype=np.uint8)
    for x, component_idx in enumerate(large):
        color = x + 1
        black_img1[labels == component_idx] = color

    x = []
    y = []
    w = []
    h = []

    for i in large:
        x.append(stats[i, 0])
        y.append(stats[i, 1])
        w.append(stats[i, 2])
        h.append(stats[i, 3])


    full_imgs = []
    real_imgs = []

    for i in range(len(large)):
        full_imgs.append(black_img1[y[i]:y[i]+h[i], x[i]:x[i]+w[i]])
        real_imgs.append(img[y[i]:y[i]+h[i], x[i]:x[i]+w[i]])

    return(full_imgs, real_imgs, object_presence, x, y, w, h, black_img1)

In [ ]:
def small_plant_segmanter(predicted_root, img, object_presence, predicted_root_2):
    """
    Segments small plants in an image based on a binary mask and predefined zones.

    This function focuses on segmenting small plants within specific zones of a binary mask. 
    It first applies a mask to preserve only the specified zones in the predicted_root image. 
    Then, it identifies and segments the largest object in each of these zones. The function 
    also creates visualizations showing the segmentation results.

    Parameters:
    predicted_root (ndarray): A binary mask image, with 1 representing the presence of an object (plant).
    img (ndarray): The original image corresponding to the binary mask.
    object_presence (ndarray): A binary array indicating the presence of objects in each region.

    Returns:
    tuple: A tuple containing:
        - full_imgs (list of ndarray): Segmented objects from the binary mask within the specified zones.
        - real_imgs (list of ndarray): Corresponding segmented objects from the original image within the specified zones.
        - object_presence (ndarray): Updated binary array indicating the presence of objects in each region after segmentation.
        - x (list of int): The x-coordinate of the top-left corner of each segmented object.
        - y (list of int): The y-coordinate of the top-left corner of each segmented object.
        - w (list of int): The width of each segmented object.
        - h (list of int): The height of each segmented object.

    Note:
    - The function defines specific zones (preserved areas) for segmentation.
    - Uses `cv2.connectedComponentsWithStats` for segmentation.
    - Visualization is created using `matplotlib.pyplot`.
    - The function assumes a minimum area threshold for object segmentation.
    """
 
    preserve_zones = [
        (450, 550, 320, 400),     # Zone 1
        (390, 500, 850, 950),     # Zone 2
        (390, 500, 1350, 1450),   # Zone 3
        (390, 500, 1900, 2000),   # Zone 4
        (390, 500, 2400, 2500)    # Zone 5
    ]

    mask = np.zeros_like(predicted_root, dtype=bool)

    for x_start, x_end, y_start, y_end in preserve_zones:
        mask[x_start:x_end, y_start:y_end] = True

    predicted_root[~mask] = 0



    label_count, labels, stats, centroids = cv2.connectedComponentsWithStats(predicted_root)

    large = []

    largest_area_1 = 0
    largest_area_2 = 0
    largest_area_3 = 0
    largest_area_4 = 0
    largest_area_5 = 0

    largest_label_1 = -1
    largest_label_2 = -1
    largest_label_3 = -1
    largest_label_4 = -1
    largest_label_5 = -1

    object_presence = np.zeros(5)



    for x in range(1, label_count):
        if (stats [x, cv2.CC_STAT_AREA] > 7):
            if ( stats[x, cv2.CC_STAT_LEFT] < 600
                ):
                area_1 = stats[x, cv2.CC_STAT_AREA]

                if area_1 > largest_area_1:
                    largest_area_1 = area_1
                    largest_label_1 = x

            if (
                stats[x, cv2.CC_STAT_LEFT] > 600
                and stats[x, cv2.CC_STAT_LEFT] < 1200
                ):
            
                area_2 = stats[x, cv2.CC_STAT_AREA]
                if area_2 > largest_area_2:
                    largest_area_2 = area_2
                    largest_label_2 = x


            if (
                stats[x, cv2.CC_STAT_LEFT] > 1200
                and stats[x, cv2.CC_STAT_LEFT] < 1600
            ):
                area_3 = stats[x, cv2.CC_STAT_AREA]
                if area_3 > largest_area_3:
                    largest_area_3 = area_3
                    largest_label_3 = x

            if (
                stats[x, cv2.CC_STAT_LEFT] > 1600
                and stats[x, cv2.CC_STAT_LEFT] < 2100
            ):
                area_4 = stats[x, cv2.CC_STAT_AREA]
                if area_4 > largest_area_4:
                    largest_area_4 = area_4
                    largest_label_4 = x

            if (
                stats[x, cv2.CC_STAT_LEFT] > 2100
            ):
                area_5 = stats[x, cv2.CC_STAT_AREA]
                if area_5 > largest_area_5:
                    largest_area_5 = area_5
                    largest_label_5 = x


    if largest_label_1 != -1:
        large.append(largest_label_1)
        object_presence[0] = 1

    if largest_label_2 != -1:
        large.append(largest_label_2)
        object_presence[1] = 1
        
    if largest_label_3 != -1:
        large.append(largest_label_3)
        object_presence[2] = 1
    
    if largest_label_4 != -1:
        large.append(largest_label_4)
        object_presence[3] = 1

    if largest_label_5 != -1:
        large.append(largest_label_5)
        object_presence[4] = 1

    black_img1 = np.zeros_like(labels, dtype=np.uint8)
    for x, component_idx in enumerate(large):
        color = x + 1
        predicted_root_2[labels == component_idx] = color
        black_img1[labels == component_idx] = color

    x = []
    y = []
    w = []
    h = []

    for i in large:
        x.append(stats[i, 0])
        y.append(stats[i, 1])
        w.append(stats[i, 2])
        h.append(stats[i, 3])


    full_imgs = []
    real_imgs = []


    for i in range(len(large)):
        full_imgs.append(black_img1[y[i]:y[i]+h[i], x[i]:x[i]+w[i]])
        real_imgs.append(img[y[i]:y[i]+h[i], x[i]:x[i]+w[i]])

    return(full_imgs, real_imgs, object_presence, x, y, w, h, predicted_root_2)

In [ ]:
def small_pipeline(img, root_imgs, real_imgs, object_presence, x, y, w, h, predicted_root):
    """
    Executes a small-scale image processing pipeline on a given image.

    This function takes an image and applies a series of image processing steps. It first predicts the root presence using an image predictor, and then segments small plants from the image. If objects (plants) are detected, it updates the provided lists with information about these objects, such as their positions and dimensions.

    Parameters:
    - img (array-like): The image to be processed.
    - root_imgs (list): A list of images of roots. This list gets updated with new root images from `img`.
    - real_imgs (list): A list of real images. This list gets updated with new real images from `img`.
    - object_presence (list): A list indicating the presence of an object in each corresponding image. This list gets updated based on object detection in `img`.
    - x (list): A list of x-coordinates of the detected objects. This list gets updated with new coordinates from `img`.
    - y (list): A list of y-coordinates of the detected objects. This list gets updated with new coordinates from `img`.
    - w (list): A list of widths of the detected objects. This list gets updated with new widths from `img`.
    - h (list): A list of heights of the detected objects. This list gets updated with new heights from `img`.

    Returns:
    Tuple of (root_imgs, real_imgs, object_presence, x, y, w, h): Updated lists containing the image data and object detection details.

    Note:
    The function relies on external functions `img_predictor` and `small_plant_segmanter`, which are not defined within this function. Ensure these functions are available in the runtime environment for correct operation.
    """
    
    predicted_root_1, _ = img_predictor(img=img ,root_thr=0.001, model_type=2)
    root_imgs_small, real_imgs_small, object_presence_small, x_small,y_small,w_small,h_small, predicted_root = small_plant_segmanter(predicted_root_1, img, object_presence, predicted_root)

    if np.any(object_presence_small == 1):
        object_presence_small = np.array(object_presence_small)

        indices = np.where(object_presence_small == 1)[0]
        for index, value in enumerate(indices):
            if object_presence[value] == 0:
                object_presence[value] = object_presence_small[value]
                root_imgs.insert(value, root_imgs_small[index])
                real_imgs.insert(value, real_imgs_small[index])
                x.insert(value, x_small[index])
                y.insert(value, y_small[index])
                w.insert(value, w_small[index])
                h.insert(value, h_small[index])
    return root_imgs, real_imgs, object_presence, x,y,w,h, predicted_root

In [ ]:
def root_length_calc(image, real_img):
    """
    Calculates the length of the longest root in an image using its skeleton representation.

    The function performs the following steps:
    1. Skeletonize the input image to obtain a simpler representation of the root structure.
    2. Summarize the skeleton to identify individual branches.
    3. Create a graph representation of the skeleton branches.
    4. Find the longest path in the graph, which corresponds to the longest root.
    5. Draw circles on the starting and ending points of the longest root on a copy of the original image.
    6. Calculate the length of the longest root.
    7. Filter and draw the longest root path on a blank or original image.

    Parameters:
    image (numpy.ndarray): A binary image containing the root structure to be analyzed.
    real_img (numpy.ndarray): The original image (colored or grayscale) on which the root structure is to be drawn.

    Returns:
    tuple: A tuple containing the following:
        - An image with circles drawn at the start and end points of the longest root.
        - The length of the longest root.
        - The skeletonized image of the root structure.
        - x and y coordinates of the start point of the longest root.
        - x and y coordinates of the end point of the longest root.
        - An image showing the longest root path.

    Note:
    The function relies on the following libraries: OpenCV, NetworkX, and NumPy. Ensure these are installed and imported as cv2, nx, and np, respectively.
    """
    skeleton = skeletonize(image)
    skeleton_branch = summarize(Skeleton(skeleton))
    G = nx.from_pandas_edgelist(skeleton_branch, source='node-id-src', target='node-id-dst', edge_attr='branch-distance')

    x_start = []
    y_start = []
    x_end = []
    y_end = []

    branches = len(skeleton_branch)
    im_with_circles = real_img.copy()
    end_root_index = skeleton_branch["node-id-dst"].nsmallest(branches).iloc[-1]

    for i in range(branches):
        if i == 0:
            x_start = skeleton_branch["image-coord-src-0"][i]
            y_start = skeleton_branch["image-coord-src-1"][i]
            im_with_circles = cv2.circle(im_with_circles, (y_start, x_start), 10, (0, 255, 0), 2)
            start_node = skeleton_branch["node-id-src"][i]
            if branches == 1:
                x_end = skeleton_branch["image-coord-dst-0"][i]
                y_end = skeleton_branch["image-coord-dst-1"][i]
                im_with_circles = cv2.circle(im_with_circles, (y_end, x_end), 10, (0, 0, 0), 2)
                end_node = skeleton_branch["node-id-dst"][i]
        else:
            if skeleton_branch["node-id-dst"][i] == end_root_index:
                x_end = skeleton_branch["image-coord-dst-0"][i]
                y_end = skeleton_branch["image-coord-dst-1"][i]
                im_with_circles = cv2.circle(im_with_circles, (y_end, x_end), 10, (0, 0, 0), 2)
                end_node = skeleton_branch["node-id-dst"][i]
    
    root_len = nx.dijkstra_path_length(G, end_node, start_node, weight='branch-distance')
    shortest_path = nx.dijkstra_path(G, source=start_node, target=end_node, weight='branch-distance')
    
    shortest_edges = [(shortest_path[i], shortest_path[i+1]) for i in range(len(shortest_path)-1)]
    filtered_branches = skeleton_branch[skeleton_branch.apply(lambda row: (row['node-id-src'], row['node-id-dst']) in shortest_edges, axis=1)]
    
    filtered_skeleton_image = np.zeros_like(real_img)

    for index, row in filtered_branches.iterrows():
        src = (int(row['image-coord-src-1']), int(row['image-coord-src-0']))  # y, x for source
        dst = (int(row['image-coord-dst-1']), int(row['image-coord-dst-0']))  # y, x for destination
        filtered_skeleton_image = cv2.line(filtered_skeleton_image, src, dst, color=(255, 255, 255), thickness=6)

    return im_with_circles, root_len, skeleton, x_start, y_start, x_end, y_end, filtered_skeleton_image

In [ ]:
def img_feeder(img):
    """
    Processes an image to identify root structures, calculate their lengths, and visualize the results.

    This function performs several steps to analyze root structures in an input image:
    1. Depending on the size of the input image, it either fixes errors or crops the image.
    2. Predicts root locations in the processed image.
    3. Segments the image to isolate individual roots and their corresponding real images.
    4. If necessary, applies a smaller pipeline for further processing.
    5. For each detected root, calculates its length and generates a visual representation of the root skeleton.
    6. Overlays the skeleton on the original image, highlighting the start and end points of each root.

    Parameters:
    img (numpy.ndarray): The input image to be processed. It should be a grayscale or colored image of root structures.

    Returns:
    tuple: A tuple containing the following:
        - A list of images with circles drawn at the start and end points of each root.
        - A list of the lengths of each identified root.
        - An image with the combined visualizations of all roots, their skeletons, and start/end points marked.

    Note:
    This function depends on several other functions (`error_fix`, `crop_img`, `img_predictor`, `segmanter`, `small_pipeline`, `root_length_calc`) and external libraries such as OpenCV and NumPy. Ensure all dependent functions and libraries are available and properly imported.
    """
    if img.shape[0] < 3000:
        cropped_img = error_fix(img)
    else:
        cropped_img = crop_img(img)
    predicted_root, output_img = img_predictor(cropped_img, root_thr=0.05, model_type=2)
    root_imgs, real_imgs, object_presence, x, y, w, h, predicted_root = segmanter(predicted_root, output_img)

    if np.any(object_presence == 0):
        root_imgs, real_imgs, object_presence, x, y, w, h, predicted_root = small_pipeline(cropped_img, root_imgs, real_imgs, object_presence, x, y, w, h, predicted_root)

    cirles = []
    root_lens = []
    skeletons = []
    filtered_skeleton_images = []
    
    end_img = output_img.copy()
    red_color = (0, 0, 255)
    end_img = cv2.cvtColor(end_img, cv2.COLOR_GRAY2BGR)
 
    i = 0
    for object in object_presence:
        if object == 1:
            circle, root_len, skeleton, y_s, x_s, y_e, x_e, filtered_skeleton_image = root_length_calc(root_imgs[i], real_imgs[i])
            cirles.append(circle)
            skeletons.append(skeleton)
            root_lens.append(root_len)
            filtered_skeleton_images.append(filtered_skeleton_image)

            x_start = x[i] + x_s
            y_start = y[i] + y_s
            x_end = x[i] + x_e
            y_end = y[i] + y_e

            filtered_skeleton_image_color = cv2.cvtColor(filtered_skeleton_image, cv2.COLOR_GRAY2BGR)
            other_image_color = cv2.cvtColor(real_imgs[i], cv2.COLOR_GRAY2BGR)
            filtered_skeleton_image_color[filtered_skeleton_image == 255] = red_color

            combined_image = cv2.add(filtered_skeleton_image_color, other_image_color)
            end_img[y[i]:y[i]+h[i], x[i]:x[i]+w[i]] = combined_image
            end_img = cv2.circle(end_img, (x_start, y_start), 50, (0, 255, 0), 6)  
            end_img = cv2.circle(end_img, (x_end, y_end), 50, (0, 0, 0), 6)  
            i += 1
        else:
            root_lens.append(0)
    return cirles, root_lens, end_img, predicted_root

In [ ]:
def folder_handler(dir):
    """
    Processes all image files in a specified directory for root structure analysis.

    This function iterates through each image file in the given directory, analyzes the root structures in each image,
    and compiles the results. It handles images with specific file extensions, reads them, and uses the `img_feeder`
    function to process each image.

    Steps:
    1. Sorts the files in the directory based on a naming convention.
    2. For each sorted image file, reads the image and processes it using `img_feeder`.
    3. Collects and aggregates the results from all images.

    Parameters:
    dir (str): The directory path containing the image files to be processed.

    Returns:
    tuple: A tuple containing the following:
        - A list of images with visualized root structures for each file.
        - A list of root lengths for each root identified across all files.
        - A list of end images with combined visualizations for each file.

    Note:
    The function expects image files to follow a specific naming convention for sorting (e.g., based on a number in the filename).
    Supported file formats are .tif and .png. This function depends on the `img_feeder` function and external libraries like OpenCV and os.
    """
    circle_imgs = []
    root_lens = []
    end_imgs = []
    data = []
    i = 1
    #file_names_sorted = sorted(os.listdir(dir), key=lambda x: int(x.split('_')[2].split('.')[0]))
    file_names_sorted = os.listdir(dir)
    os.makedirs("outputs", exist_ok=True)
    for img_file in file_names_sorted:
        if img_file.endswith(".tif") or img_file.endswith(".png"):
            img = cv2.imread(f"{dir}/{img_file}", 0)
            try:
                circle_img, root_len, end_img, predicted_root = img_feeder(img)
            except Exception as e:
                continue
            circle_imgs.extend(circle_img)
            root_lens.extend(root_len)
            end_imgs.append(end_img)
            for j, length in enumerate(root_len):
                row_value = f"test_image_{i}_plant_{j + 1}"
                data.append([row_value, length])
            cv2.imwrite(f"outputs/{img_file}", predicted_root)
            i += 1
    df = pd.DataFrame(data, columns=['Plant ID', 'Length (px)'])
    df.to_csv(Save_results_as, index=False)
    return(circle_imgs, root_lens, end_imgs)

In [ ]:
def smape(actual, forecast):
    if np.array_equal(actual, forecast):
        print("The arrays are identical.")
        return 0
    else:
        denominator = (np.abs(actual) + np.abs(forecast)) / 2
        diff = np.abs(forecast - actual) / denominator
        diff[denominator == 0] = 0.0 
        return 100 * np.mean(diff)

In [ ]:
def plotter(circle_ims):
    leng = int(len(circle_ims)/5)
    fig, ax = plt.subplots(leng, 5, dpi=250)
    x = 0
    for i in range(leng):
        ax[i, 0].imshow(circle_ims[x])
        ax[i, 0].axis('off')
        ax[i, 1].imshow(circle_ims[x + 1])
        ax[i, 1].axis('off')
        ax[i, 2].imshow(circle_ims[x + 2])
        ax[i, 2].axis('off')
        ax[i, 3].imshow(circle_ims[x + 3])
        ax[i, 3].axis('off')
        ax[i, 4].imshow(circle_ims[x + 4])
        ax[i, 4].axis('off')
        x += 5
    plt.show()


In [ ]:
circles, root_lens, end_imgs = folder_handler(Input_dataset)